In [1]:
!pip install -q torch transformers transformers langchain sentence-transformers tqdm openpyxl openai pandas datasets langchain-community ragatouille

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-cohere 0.1.9 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.1.52 which is incompatible.
langchain-experimental 0.0.62 requires langchain-community<0.3.0,>=0.2.6, but you have langchain-community 0.0.38 which is incompatible.
langchain-experimental 0.0.62 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.1.52 which is incompatible.
langchain-groq 0.1.9 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.1.52 which is incompatible.
langchain-openai 0.1.20 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.1.52 which is incompatible.


In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import json
import datasets

pd.set_option("display.max_colwidth", None)

In [5]:
!pip install ipywidgets

In [35]:
from huggingface_hub import notebook_login
# 
notebook_login()    

In [28]:
from datasets import load_dataset

# Tải tập dữ liệu
ds_legalRAG = load_dataset("sailor2/Vietnamese_RAG", 'LegalRAG')

# Xáo trộn tập train và chỉ lấy 2 dòng ngẫu nhiên
ds_legalRAG_train_random_2 = ds_legalRAG['train'].shuffle(seed=42).select(range(2))

# In ra 2 dòng ngẫu nhiên
for row in ds_legalRAG_train_random_2:
    print(row)

{'question': 'Khám bệnh, chữa bệnh đối với phạm nhân theo Luật thi hành án hình sự mới nhất được quy định như thế nào?', 'answer': 'Phạm nhân được hưởng chế độ phòng, chống dịch bệnh. Trại giam, trại tạm giam, cơ quan thi hành án hình sự Công an cấp huyện phối hợp với cơ sở khám bệnh, chữa bệnh cấp huyện hoặc bệnh viện quân đội nơi trại giam, trại tạm giam, cơ quan thi hành án hình sự Công an cấp huyện đóng tổ chức khám sức khỏe cho phạm nhân. Phạm nhân bị ốm, bị thương tích thì được khám bệnh, chữa bệnh và điều trị tại cơ sở y tế của trại giam, trại tạm giam hoặc tại cơ sở khám bệnh, chữa bệnh của Nhà nước nơi gần nhất. Trường hợp phạm nhân bị bệnh nặng hoặc thương tích vượt quá khả năng điều trị của cơ sở khám bệnh, chữa bệnh đó thì được chuyển đến cơ sở khám bệnh, chữa bệnh tuyến trên để điều trị; trại giam, trại tạm giam, cơ quan thi hành án hình sự Công an cấp huyện phải thông báo cho thân nhân hoặc đại diện của người đó biết để phối hợp chăm sóc, điều trị. Chế độ ăn, cấp phát thu

In [49]:
from huggingface_hub import InferenceClient


repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)


def call_llm(inference_client: InferenceClient, prompt: str):
    response = inference_client.post(
        json={
            "inputs": prompt,
            "parameters": {"max_new_tokens": 1000},
            "task": "text-generation"
        },
    )
    return json.loads(response.decode())[0]["generated_text"]


call_llm(llm_client, "Hãy trình bày cho tôi cách tính diện tích hình bình hành và trả lời bằng tiếng việt")

'Hãy trình bày cho tôi cách tính diện tích hình bình hành và trả lời bằng tiếng việt.\n\nTo calculate the area of a rectangle, you need to multiply its length by its width. In Vietnamese, this can be expressed as: "Diện tích hình bình hành là chiều dài nhân chiều rộng."\n\nFor example, if a rectangle has a length of 5 units and a width of 3 units, you would calculate the area as follows:\n\nDiện tích = Chiều dài x Chiều rộng\nDiện tích = 5 x 3\nDiện tích = 15\n\nTherefore, the area of the rectangle is 15 square units.'

json response format follow here
{
        {
            "Groundedness": {
                "Rating": "",
                "Reason": ""
            }
        },
        {
            "Relevance": {
                "Rating": "",
                "Reason": ""
            }
        },
        {
            "Context-Independence": {
                "Rating": "",
                "Reason": ""
            }
        }
    
}

In [29]:
optimizer_prompt = """
Task: Evaluate the given question based on three criteria: groundedness, relevance, and context-independence. Your task is to provide a detailed assessment for each criterion and then give an overall rating.

Instructions:
Groundedness: Assess how well the question can be answered using only the provided context. Rate on a scale of 1 to 5, where 1 means the question is not answerable at all with the given context, and 5 means the question is clearly and unambiguously answerable.
Relevance: Evaluate how useful the question is for someone seeking legal information. Rate on a scale of 1 to 5, where 1 means the question is not useful at all, and 5 means the question is extremely useful.
Context-Independence: Determine how independent the question is from additional context. Rate on a scale of 1 to 5, where 1 means the question depends heavily on additional context to be understood, and 5 means the question is clear on its own.
Provide your assessment in the following format:
Answer:::

Groundedness:
Evaluation: (Your rationale for the rating, as a text. Reply in Vietnamese)
Rating: (Your rating, as a number between 1 and 5)
Relevance:
Evaluation: (Your rationale for the rating, as a text. Reply in Vietnamese)
Rating: (Your rating, as a number between 1 and 5)
Context-Independence:
Evaluation: (Your rationale for the rating, as a text. Reply in Vietnamese)
Rating: (Your rating, as a number between 1 and 5)
Overall Rating: (Your overall rating based on the three criteria, as a number between 1 and 5)
Now here are the question and context:

Question: {question}\n
Context: {context}\n
Reply in Vietnamese
Provide your Answer. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Answer:::


"""

In [60]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (The reason you evaluate and respond in Vietnamese)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide an 'overall assessment' representing how useful this question is to a user seeking legal information.
Give your answer on a scale of 1 to 5, where 1 means the question is not useful at all, and 5 means the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (The reason you evaluate and respond in Vietnamese)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For example, if the question refers to a specific context, such as 'in this situation' or 'in the document,' the rating should be 1. 
A question can contain technical terms or legal jargon and still be rated a 5: it just needs to be clear to a user seeking legal information.

For example, "In this case, which law applies?" should receive a 1 because it implicitly references a specific context, making the question not context-independent.

Provide your answer as follows:

Answer:::
Evaluation:   (The reason you evaluate and respond in Vietnamese)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [11]:
# Thử theo phương pháp người ta 
print("Generating critique for each QA couple...")

outputs = []
for output in tqdm(ds_legalRAG_train_random_2):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(context=output["context"][0], question=output["question"]),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
        outputs.append(output)
    except Exception as e:
        continue

with open('output_results.json', 'w', encoding='utf-8') as f:
    json.dump(outputs, f, ensure_ascii=False, indent=4)

print("Đã lưu kết quả vào file output_results.json.")

Generating critique for each QA couple...


NameError: name 'ds_legalRAG_train_random_2' is not defined

In [15]:
!pip install groq

## load data jsonl của legal_zalo 2022

In [1]:
import json
import random
def getdata(num: int = None):
    samples = []
    
    with open('/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/data/legal_zalo.json', 'r') as file:
        corpus = json.load(file)
        for data in corpus:
            samples.append(data)
    
    # If num is not provided or is None, return all samples
    if num is None:
        return corpus
    else:
        # Return a random sample of `num` records
        return random.sample(samples, min(num, len(samples)))
    

In [2]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (The reason you evaluate and respond in Vietnamese)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide an 'overall assessment' representing how useful this question is to a user seeking legal information.
Give your answer on a scale of 1 to 5, where 1 means the question is not useful at all, and 5 means the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (The reason you evaluate and respond in Vietnamese)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independant this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For example, if the question refers to a specific context, such as 'in this situation' or 'in the document,' the rating should be 1. 
A question can contain technical terms or legal jargon and still be rated a 5: it just needs to be clear to a user seeking legal information.

For example, "In this case, which law applies?" should receive a 1 because it implicitly references a specific context, making the question not context-independent.

Provide your answer as follows:

Answer:::
Evaluation:   (The reason you evaluate and respond in Vietnamese)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

### routing api 

In [3]:
import os
import json
from tqdm.notebook import tqdm
from groq import Groq
from dotenv import load_dotenv

load_dotenv()
api_keys = [
    os.environ.get("GROQ_API_KEY1")
    # os.environ.get("GROQ_API_KEY2")
    # os.environ.get("GROQ_API_KEY3"),
    # os.environ.get("GROQ_API_KEY4")
]

api_key_index = 0

def get_chat_completion(prompt: str) -> str:
    global api_key_index  # Để cập nhật chỉ số toàn cục

    # Lấy API key hiện tại
    api_key = api_keys[api_key_index]
    # print(f'api key sử dụng {api_key}')
    # Cập nhật chỉ số để chuyển đổi giữa các API key
    api_key_index = (api_key_index + 1) % len(api_keys)

    # Khởi tạo Groq client với API key hiện tại
    client = Groq(api_key=api_key)

    # Tạo yêu cầu hoàn thành chat
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    
    # Trả về nội dung của phản hồi
    return chat_completion.choices[0].message.content




print("Đang tạo đánh giá cho từng cặp Hỏi-Đáp...")
samples = getdata()  # Giả sử getdata() là một hàm trả về danh sách các mẫu

sub_samples = samples[2375:]
print(sub_samples[0])


with open('groq_output.json', 'a', encoding='utf-8') as f:
    for sample in tqdm(sub_samples):
        evaluations = {
            "groundedness": get_chat_completion(
                question_groundedness_critique_prompt.format(context=sample["context"], question=sample["question"]),
            ),
            "relevance": get_chat_completion(
                question_relevance_critique_prompt.format(question=sample["question"]),
            ),
            "standalone": get_chat_completion(
                question_standalone_critique_prompt.format(question=sample["question"]),
            ),
        }
        try:
            for criterion, evaluation in evaluations.items():
                score, eval = (
                    int(evaluation.split("Total rating: ")[-1].strip()),
                    evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
                )
                sample.update(
                    {
                        f"{criterion}_score": score,
                        f"{criterion}_eval": eval,
                    } 
                )   
            
            # Ghi mẫu đã được xử lý vào file JSON
            f.write(json.dumps(sample, ensure_ascii=False, indent=4) + ",\n")
        except Exception as e:
            continue

print("Đã lưu kết quả vào file groq_output.json.")


Đang tạo đánh giá cho từng cặp Hỏi-Đáp...
{'question': 'Trường hợp nào chạy xe máy ban ngày nhưng vẫn phải bật đèn xe?', 'answer': 'Trường hợp nào là khi sương mù, thời tiết xấu hạn chế tầm nhìn.', 'context': '1. Phạt tiền từ 100.000 đồng đến 200.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:\na) Không chấp hành hiệu lệnh, chỉ dẫn của biển báo hiệu, vạch kẻ đường, trừ các hành vi vi phạm quy định tại điểm c, điểm đ, điểm e, điểm h khoản 2; điểm d, điểm g, điểm i, điểm m khoản 3; điểm a, điểm b, điểm c, điểm d, điểm e khoản 4; khoản 5; điểm b khoản 6; điểm a, điểm b khoản 7; điểm d khoản 8 Điều này;\nb) Không có báo hiệu xin vượt trước khi vượt;\nc) Không giữ khoảng cách an toàn để xảy ra va chạm với xe chạy liền trước hoặc không giữ khoảng cách theo quy định của biển báo hiệu “Cự ly tối thiểu giữa hai xe”;\nd) Chuyển hướng không nhường quyền đi trước cho: Người đi bộ, xe lăn của người khuyết tật qua đường tại nơi có vạch kẻ đường dành cho người đi

  0%|          | 0/103 [00:00<?, ?it/s]

Đã lưu kết quả vào file groq_output.json.


In [12]:
import os
from groq import Groq

from dotenv import load_dotenv
load_dotenv()

def get_chat_completion(prompt: str) -> str:
    # Initialize the Groq client with the API key from environment variables
    client = Groq(api_key=os.environ.get("GROQ_API_KEY5"))
    
    # Create a chat completion request
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    
    # Return the content of the response
    return chat_completion.choices[0].message.content
print("Generating critique for each QA couple...")
samples = getdata(1)

outputs = []
for sample in tqdm(samples):
    print(sample["question"])
    evaluations = {
        "groundedness": get_chat_completion(
            question_groundedness_critique_prompt.format(context=sample["context"], question=sample["question"]),
        ),
        "relevance": get_chat_completion(
            question_relevance_critique_prompt.format(question=sample["question"]),
        ),
        "standalone": get_chat_completion(
            question_standalone_critique_prompt.format(question=sample["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            sample.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
        print(sample)
        outputs.append(sample)
    except Exception as e:
        continue

with open('groq_output.json', 'w', encoding='utf-8') as f:
    json.dump(outputs, f, ensure_ascii=False, indent=4)

print("Đã lưu kết quả vào file groq_output.json.")

Generating critique for each QA couple...


  0%|          | 0/1 [00:00<?, ?it/s]

Thẩm quyền thành lập, mở rộng khu kinh tế được quy định như thế nào?


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

### Chuyển jsonl thành json

In [52]:
import json

# Define the input and output file paths
input_file = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/modified_data_BKAI.jsonl'
output_file = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/modified_data_BKAI.json'

# Initialize an empty list to hold all the samples
data = []

# Read the JSONL file and collect samples
with open(input_file, 'r', encoding='utf-8') as infile:
    for line in infile:
        # Parse each line as a JSON object and add it to the list
        sample = json.loads(line)
        data.append(sample)

# Write the list of samples to a JSON file
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(data, outfile, ensure_ascii=False, indent=4)

print("Conversion complete.")


Conversion complete.


### đánh giá và lọc dữ liệu

In [8]:
import json


eval_legal_data = 'eval-data/legal_eval.json'
eval_bkai_data = 'eval-data/bkai_eval.json'


def getdata(file_path : str):
    samples = []
    with open(file_path, 'r') as file:
        corpus = json.load(file)
        for data in corpus:
            samples.append(data)
        return corpus
data = getdata(eval_legal_data)


In [9]:


import pandas as pd
import datasets
pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame(data)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "context", 
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

Evaluation dataset before filtering:


question  \
0                                                         Công an xã xử phạt lỗi không mang bằng lái xe có đúng không?   
1                          Phải thực hiện thao tác nạp mẫu vào bình chứa và xử lý mẫu sơ bộ bằng hóa chất như thế nào?   
2               Tử tù chết sau khi tiêm thuốc độc mà không có thân nhân nhận xác thì được hỗ trợ mai táng như thế nào?   
3     Trách nhiệm của Bộ Khoa học và Công nghệ về quản lý và phát triển công nghiệp an ninh được quy định như thế nào?   
4                                                   Mức phạt khi sử dụng trái phép lòng đường đô thị, hè phố để rửa xe   
...                                                                                                                ...   
2142                        Việc đàm phán, hoàn thiện hợp đồng đối với dự án đầu tư có sử dụng đất dựa trên cơ sở nào?   
2143         Thông tin giá thị trường của các yếu tố đầu vào để xác định chỉ số giá xây dựng phải thỏa mãn yêu cầu gì?   
2144     Mức phạt đối với hành vi không niêm yết công khai giá mua bán hàng dự trữ quốc gia được quy định như thế nào?   
2145                           Con dấu của Cơ quan quản lý thi hành án hình sự thuộc Bộ Công an được quy định thế nào?   
2146                                                                                Đánh giá hiện trạng vùng bờ là gì?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               answer  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Không đúng. Công an xã chỉ được tuần tra, kiểm soát và xử lý các hành vi vi phạm trật tự, an toàn giao thông như điều khiển xe mô tô, xe gắn máy không đội mũ bảo hiểm, chở quá số người quy định, chở hàng hóa cồng kềnh; đỗ xe ở lòng đường trái quy định; điều khiển phương tiện phóng nhanh, lạng lách, đánh võng, tháo ống xả, không có gương chiếu hậu hoặc chưa đủ tuổi điều khiển phương tiện theo quy định của pháp luật và các hành vi vi phạm hành lang an toàn giao thông đường bộ như họp chợ dưới lòng đường, lấn chiếm hành lang an toàn giao thông.   
1     Phải dùng ca để lấy mẫu từ dụng cụ chứa trung gian nạp vào bình chứa. Mức độ đầy vơi khi nạp mẫu vào bình chứa phải căn cứ vào quy định đối với từng loại thông số phân tích. Với những bình mẫu phải cho hóa chất bảo quản thì chỉ nạp gần đủ, rồi thực hiện bước nạp hóa chất bảo quản mẫu (xử lý mẫu sơ bộ bằng hóa chất).

Final evaluation dataset:


,question,answer,groundedness_score,relevance_score,standalone_score
3,Trách nhiệm của Bộ Khoa học và Công nghệ về quản lý và phát triển công nghiệp an ninh được quy định như thế nào?,"Bộ Khoa học và Công nghệ có trách nhiệm chủ trì, phối hợp, hướng dẫn Bộ Công an và các tổ chức, cá nhân thực hiện các chương trình, nhiệm vụ nghiên cứu, ứng dụng khoa học công nghệ phục vụ xây dựng, phát triển công nghiệp an ninh, tập trung nghiên cứu, ứng dụng, cập nhật, phát triển sản phẩm mới, sản phẩm chuyên dụng công nghệ cao đáp ứng yêu cầu ngày càng cao của nhiệm vụ bảo vệ an ninh quốc gia, bảo đảm trật tự, an toàn xã hội trong tình hình mới; phối hợp, hướng dẫn, kiểm tra công tác chuyển giao công nghệ, giải mã công nghệ; ban hành quy chuẩn kỹ thuật quốc gia về các sản phẩm công nghiệp an ninh chuyên dụng và việc huy động các chuyên gia, nhà khoa học tham gia nghiên cứu, phát triển công nghiệp an ninh.",5,5,5
4,"Mức phạt khi sử dụng trái phép lòng đường đô thị, hè phố để rửa xe","Phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng đối với cá nhân, từ 8.000.000 đồng đến 12.000.000 đồng đối với tổ chức.",5,4,5
5,Hai công ty con chung công ty mẹ có được tham dự gói thầu của nhau không?,"Nếu sản phẩm, dịch vụ thuộc gói thầu là ngành nghề sản xuất kinh doanh chính của tập đoàn và là đầu ra của công ty này, đồng thời là đầu vào của công ty kia trong tập đoàn thì hai công ty con đó được phép tham dự quá trình lựa chọn nhà thầu của nhau.",4,4,5
6,Mức phạt khi không thực hiện đúng quy trình tuyển sinh theo quy định của pháp luật là bao nhiêu?,Phạt tiền từ 20.000.000 đồng đến 30.000.000 đồng.,5,5,5
8,Dự án thuộc ngành nghề ưu đãi đầu tư tại địa bàn khó khăn được hưởng ưu đãi như thế nào?,"Dự án đầu tư thuộc ngành, nghề ưu đãi đầu tư thực hiện tại địa bàn có điều kiện kinh tế - xã hội khó khăn được hưởng ưu đãi đầu tư như quy định đối với dự án đầu tư thuộc địa bàn có điều kiện kinh tế - xã hội đặc biệt khó khăn.",5,4,5
...,...,...,...,...,...
2141,Mức phạt khi thi công trên đường bộ đang khai thác không treo biển báo thông tin công trình,"Cảnh cáo hoặc phạt tiền từ 250.000 đồng đến 500.000 đồng đối với cá nhân, từ 500.000 đồng đến 1.000.000 đồng đối với tổ chức.",5,5,5
2142,"Việc đàm phán, hoàn thiện hợp đồng đối với dự án đầu tư có sử dụng đất dựa trên cơ sở nào?","Việc đàm phán, hoàn thiện hợp đồng đối với dự án đầu tư có sử dụng đất dựa trên cơ sở sau: Báo cáo đánh giá hồ sơ dự thầu; Hồ sơ dự thầu và các tài liệu làm rõ, sửa đổi hồ sơ dự thầu (nếu có) của nhà đầu tư; Hồ sơ mời thầu.",5,5,4
2144,Mức phạt đối với hành vi không niêm yết công khai giá mua bán hàng dự trữ quốc gia được quy định như thế nào?,"Mức phạt đối với hành vi không thông báo hoặc niêm yết công khai về giá mua, bán hàng dự trữ quốc gia được quy định là phạt tiền từ 1.000.000 đồng đến 3.000.000 đồng.",5,5,5
2145,Con dấu của Cơ quan quản lý thi hành án hình sự thuộc Bộ Công an được quy định thế nào?,Con dấu của Cơ quan quản lý thi hành án hình sự thuộc Bộ Công an có tên gọi là: Bộ Công an - Cơ quan quản lý thi hành án hình sự.,4,5,5


In [35]:
print(type(eval_dataset))
fields = eval_dataset.features.keys()
print(fields)

<class 'datasets.arrow_dataset.Dataset'>
dict_keys(['question', 'answer', 'context', 'groundedness_score', 'groundedness_eval', 'relevance_score', 'relevance_eval', 'standalone_score', 'standalone_eval'])


In [10]:
import datasets
from datasets import DatasetDict


dataset_dict = DatasetDict({"train": eval_dataset})


dataset_dict.push_to_hub("Eval-RAG-Vietnamese", 'legal-data')


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/649 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tuananh18/Eval-RAG-Vietnamese/commit/3ad9e43532ab8204a62a08d570a59a4adcf0d75f', commit_message='Upload dataset', commit_description='', oid='3ad9e43532ab8204a62a08d570a59a4adcf0d75f', pr_url=None, pr_revision=None, pr_num=None)

### chuẩn hóa lại context 

In [1]:
!git clone https://huggingface.co/datasets/tuananh18/Eval-RAG-Vietnamese

Cloning into 'Eval-RAG-Vietnamese'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (13/13), 3.80 KiB | 3.80 MiB/s, done.


In [10]:
from datasets import load_dataset
import json

# Load the dataset
ds_legalRAG = load_dataset("sailor2/Vietnamese_RAG", 'LegalRAG')

# Select the 'train' split
data = ds_legalRAG['train']

# Convert the data to a list of dictionaries
data_list = [row for row in data]

# Write the data to a JSON file
with open('legal_duy.json', 'w', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)


In [8]:
import json

# Load JSON files
with open('/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/legal_data_after.json', 'r', encoding='utf-8') as f1:
    data1 = json.load(f1)

with open('/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/eval-data/legal_eval.json', 'r', encoding='utf-8') as f2:
    data2 = json.load(f2)

# Create a dictionary to map questions to contexts from file 2
context_map_2 = {entry['question']: entry['context'] for entry in data2}

# Update context in file1 based on matching questions
for entry in data1:
    question = entry['question']
    if question in context_map_2:
        entry['context'] = context_map_2[question]

# Save the updated JSON file 1
with open('file1_updated1.json', 'w', encoding='utf-8') as f1_updated:
    json.dump(data1, f1_updated, ensure_ascii=False, indent=4)
